# Connect to PowerBI Desktop sample (as is, *not extensively tested*)

## Requirements 

### Windows only

###  (Python Packages)
pip install pythonnet pandas pandas-profiling

### DevScope PowerBI Powershell module
https://github.com/DevScope/powerbi-powershell-modules
https://www.powershellgallery.com/packages/PowerBIPS/1.3.4.0

In [8]:
import clr
clr.AddReference("System.Data")
import System.Data.OleDb as ADONET
import System
import pandas as pd
import pandas_profiling
import numpy as np

In [9]:
# Juyter shell cell and DevScope powerbi-powershell
pbi_port=!powershell Get-PBIDesktopTCPPort "*Semmelweis*"

connStr = f"Provider=MSOLAP; Data Source=localhost:{pbi_port[0]}"
print(f"Connecting to {connStr}...")
connection = ADONET.OleDbConnection(connStr)
connection.Open()

Connecting to Provider=MSOLAP; Data Source=localhost:53258...


In [10]:
def get_powerbi_table(connection,table_name,sample_rows=None):
    command = connection.CreateCommand()
    if sample_rows is None:
        command.CommandText = f"evaluate '{table_name}'"
    else:
        command.CommandText = f"evaluate topn({sample_rows},'{table_name}')"
    command.CommandTimeout=120
    reader = command.ExecuteReader()
    
    schema=reader.GetSchemaTable()
    numCol = schema.Rows.Count
    columnNames = []
    for i in range(numCol):
        columnNames.append(str(schema.Rows[i][0]))

    data = []
    while reader.Read():
        row = []
        for j in range(numCol):
            val=reader[j]
            if type(val) is System.DBNull:
                val=np.NaN
            row.append(val)

        data.append(row)


    df = pd.DataFrame(data)
    
    # Convert System.DateTime values
    for i_column in df.columns[pd.DataFrame(df.iloc[0,:].map(type))[0]==System.DateTime]:
        df[i_column]=pd.to_datetime(df[i_column].astype(str),format='%d/%m/%Y %H:%M:%S')
    
    df.columns = columnNames 
    return df

In [11]:
df_datasets=get_powerbi_table(connection,"datasets")
df_datasets

,Datasets[Year],Datasets[Clinic],Datasets[Births],Datasets[Deaths],Datasets[Date],Datasets[Hospital],Datasets[Dataset],Datasets[Month],Datasets[Detail],Datasets[Mortality (base)],Datasets[Label],Datasets[Note]
0,1784,,284.0,6.0,1784-01-01,Vienna,ViennaDublin,NaN,Vienna,0.021127,1784- Vienna,No Pathological anatomy
1,1785,,899.0,13.0,1785-01-01,Vienna,ViennaDublin,NaN,Vienna,0.014461,1785- Vienna,No Pathological anatomy
2,1786,,1151.0,5.0,1786-01-01,Vienna,ViennaDublin,NaN,Vienna,0.004344,1786- Vienna,No Pathological anatomy
3,1787,,1407.0,5.0,1787-01-01,Vienna,ViennaDublin,NaN,Vienna,0.003554,1787- Vienna,No Pathological anatomy
4,1788,,1425.0,5.0,1788-01-01,Vienna,ViennaDublin,NaN,Vienna,0.003509,1788- Vienna,No Pathological anatomy
5,1789,,1246.0,7.0,1789-01-01,Vienna,ViennaDublin,NaN,Vienna,0.005618,1789- Vienna,No Pathological anatomy
6,1790,,1326.0,10.0,1790-01-01,Vienna,ViennaDublin,NaN,Vienna,0.007541,1790- Vienna,No Pathological anatomy
7,1791,,1395.0,8.0,1791-01-01,Vienna,ViennaDublin,NaN,Vienna,0.005735,1791- Vienna,No Pathological anatomy
8,1792,,1579.0,14.0,1792-01-01,Vienna,ViennaDublin,NaN,Vienna,0.008866,1792- Vienna,No Pathological anatomy
9,1793,,1684.0,44.0,1793-01-01,Vienna,ViennaDublin,NaN,Vienna,0.026128,1793- Vienna,No Pathological anatomy


In [12]:
pandas_profiling.ProfileReport(df_datasets)

Number of variables,12
Number of observations,283
Total Missing (%),5.5%
Total size in memory,26.6 KiB
Average record size in memory,96.3 B
Numeric,5
Categorical,5
Boolean,0
Date,1
Text (Unique),1
Rejected,0
